In [1]:
import os, time,json
import tmdbsimple as tmdb 
import pandas as pd
from tqdm.notebook import tqdm_notebook
FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

['.ipynb_checkpoints',
 'final_tmdb_data_2000.0.csv.gz',
 'final_tmdb_data_2001.0.csv.gz',
 'title_akas.csv.gz',
 'title_basics.csv.gz',
 'title_ratings.csv.gz',
 'tmdb_api_results_2000.0.json',
 'tmdb_api_results_2001.0.json',
 'tmdb_results_combined.csv.gz']

In [21]:
import seaborn as sns
from scipy import stats
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_transformer, make_column_selector, ColumnTransformer
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error
## fixing random for lesson generation
np.random.seed(321)

In [2]:
with open('/Users/Joene/.secret/tmdb_api.json', 'r') as f:
    login = json.load(f)
## Display the keys of the loaded dict
login.keys()

dict_keys(['api-key'])

In [3]:
tmdb.API_KEY =  login['api-key']

In [5]:
def get_movie_with_rating (movie_id):
    """Adapted from source = https://github.com/celiao/tmdbsimple"""
    # Get the movie object for the current id
    movie = tmdb.Movies(movie_id)
    
    # Save the .info .release dictionaries
    info = movie.info()
    
    releases = movie.releases()
    # Loop through countries in releases
    for c in releases['countries']:
        # if the country abbreviation == US
        if c['iso_3166_1'] == 'US':
            ## Save a "certification" key in info with the certification
            info['certification'] = c['certification']
    return info

In [6]:
test1 = get_movie_with_rating("tt0848228") #put your function name here
test1

{'adult': False,
 'backdrop_path': '/9BBTo63ANSmhC4e6r62OJFuK2GL.jpg',
 'belongs_to_collection': {'id': 86311,
  'name': 'The Avengers Collection',
  'poster_path': '/yFSIUVTCvgYrpalUktulvk3Gi5Y.jpg',
  'backdrop_path': '/zuW6fOiusv4X9nnW3paHGfXcSll.jpg'},
 'budget': 220000000,
 'genres': [{'id': 878, 'name': 'Science Fiction'},
  {'id': 28, 'name': 'Action'},
  {'id': 12, 'name': 'Adventure'}],
 'homepage': 'https://www.marvel.com/movies/the-avengers',
 'id': 24428,
 'imdb_id': 'tt0848228',
 'original_language': 'en',
 'original_title': 'The Avengers',
 'overview': 'When an unexpected enemy emerges and threatens global safety and security, Nick Fury, director of the international peacekeeping agency known as S.H.I.E.L.D., finds himself in need of a team to pull the world back from the brink of disaster. Spanning the globe, a daring recruitment effort begins!',
 'popularity': 99.992,
 'poster_path': '/RYMX2wcKCBAr24UyPD7xwmjaTn.jpg',
 'production_companies': [{'id': 420,
   'logo_path'

In [7]:
## make a movie object using the .Movies function from tmdb
movie = tmdb.Movies(603)

In [8]:
## movie objects have a .info dictionary 
info = movie.info()
info

{'adult': False,
 'backdrop_path': '/l4QHerTSbMI7qgvasqxP36pqjN6.jpg',
 'belongs_to_collection': {'id': 2344,
  'name': 'The Matrix Collection',
  'poster_path': '/bV9qTVHTVf0gkW0j7p7M0ILD4pG.jpg',
  'backdrop_path': '/bRm2DEgUiYciDw3myHuYFInD7la.jpg'},
 'budget': 63000000,
 'genres': [{'id': 28, 'name': 'Action'},
  {'id': 878, 'name': 'Science Fiction'}],
 'homepage': 'http://www.warnerbros.com/matrix',
 'id': 603,
 'imdb_id': 'tt0133093',
 'original_language': 'en',
 'original_title': 'The Matrix',
 'overview': 'Set in the 22nd century, The Matrix tells the story of a computer hacker who joins a group of underground insurgents fighting the vast and powerful computers who now rule the earth.',
 'popularity': 61.539,
 'poster_path': '/f89U3ADr1oiB1s9GkdPOEpXUk5H.jpg',
 'production_companies': [{'id': 79,
   'logo_path': '/tpFpsqbleCzEE2p5EgvUq6ozfCA.png',
   'name': 'Village Roadshow Pictures',
   'origin_country': 'US'},
  {'id': 372,
   'logo_path': None,
   'name': 'Groucho II Film

In [9]:
def write_json(new_data, filename): 
    """Appends a list of records (new_data) to a json file (filename). 
    Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/"""  
    
    with open(filename,'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        ## Choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
             file_data.append(new_data)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file)

In [10]:
YEARS_TO_GET = [2010.0,2011.0,2012.0,2013.0,2014.0,2015.0,2016.0,2017.0,2018.0,2019.0,]

In [11]:
errors = [ ]

In [12]:
# Load in the dataframe from project part 1 as basics:
basics = pd.read_csv('Data/title_basics.csv.gz')
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020.0,NaN,70,Drama
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama
3,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,NaN,100,"Comedy,Horror,Sci-Fi"
4,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002.0,NaN,126,Drama


In [13]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89149 entries, 0 to 89148
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   tconst          89149 non-null  object 
 1   titleType       89149 non-null  object 
 2   primaryTitle    89149 non-null  object 
 3   originalTitle   89149 non-null  object 
 4   isAdult         89149 non-null  int64  
 5   startYear       86611 non-null  float64
 6   endYear         0 non-null      float64
 7   runtimeMinutes  89149 non-null  int64  
 8   genres          89149 non-null  object 
dtypes: float64(2), int64(2), object(5)
memory usage: 6.1+ MB


In [14]:
# Start of OUTER loop
for YEAR in tqdm_notebook(YEARS_TO_GET, desc='YEARS', position=0):
    
    #Defining the JSON file to store results for year
    JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'
    
    # Check if file exists
    file_exists = os.path.isfile(JSON_FILE)
    
    # If it does not exist: create it
    if file_exists == False:
    # save an empty dict with just "imdb_id" to the new json file.
        with open(JSON_FILE,'w') as f:
            json.dump([{'imdb_id':0}],f)
            
    #Saving new year as the current df
    df = basics.loc[basics['startYear']==YEAR].copy()
    
    # saving movie ids to list
    movie_ids = df['tconst'].copy()
    
    # Load existing data from json into a dataframe called "previous_df"
    previous_df = pd.read_json(JSON_FILE)
    
    # filter out any ids that are already in the JSON_FILE
    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]
    

    #Get index and movie id from list
    # INNER Loop
    for movie_id in tqdm_notebook(movie_ids_to_get,
                                  desc=f'Movies from {YEAR}',
                                  position=1,
                                  leave=True):
        # Attempt to retrieve then data for the movie id
        try:
            # Retrieve then data for the movie id
            temp = get_movie_with_rating(movie_id)  
            # Append/extend results to existing file using a pre-made function
            write_json(temp,JSON_FILE)
            # Short 20 ms sleep to prevent overwhelming server
            time.sleep(0.02)
            
        except Exception as e:
            errors.append([movie_id, e])
            
    final_year_df = pd.read_json(JSON_FILE)
    final_year_df.to_csv(f"{FOLDER}final_tmdb_data_{YEAR}.csv.gz", compression="gzip", index=False)

YEARS:   0%|          | 0/10 [00:00<?, ?it/s]

Movies from 2010.0:   0%|          | 0/3848 [00:00<?, ?it/s]

Movies from 2011.0:   0%|          | 0/4224 [00:00<?, ?it/s]

Movies from 2012.0:   0%|          | 0/4510 [00:00<?, ?it/s]

Movies from 2013.0:   0%|          | 0/4702 [00:00<?, ?it/s]

...

Movies from 2015.0:   0%|          | 0/5055 [00:00<?, ?it/s]

Movies from 2016.0:   0%|          | 0/5251 [00:00<?, ?it/s]

Movies from 2017.0:   0%|          | 0/5644 [00:00<?, ?it/s]

Movies from 2018.0:   0%|          | 0/5772 [00:00<?, ?it/s]

Movies from 2019.0:   0%|          | 0/5873 [00:00<?, ?it/s]

In [15]:
print(f"- Total errors: {len(errors)}")

- Total errors: 11645


In [18]:
testdf = pd.read_csv("Data/final_tmdb_data_2019.0.csv.gz")
testdf.head()

,imdb_id,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,original_language,original_title,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,certification
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,tt0385887,0.0,/xMHeU84wLQ9uH9acJ0oPgdii9wO.jpg,NaN,26000000.0,"[{'id': 18, 'name': 'Drama'}, {'id': 53, 'name...",https://www.motherlessbrooklynfilm.com,504562.0,en,Motherless Brooklyn,...,18377736.0,145.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,NaN,Motherless Brooklyn,0.0,6.722,1273.0,R
2,tt0437086,0.0,/8RKBHHRqOMOLh5qW3sS6TSFTd8h.jpg,NaN,170000000.0,"[{'id': 28, 'name': 'Action'}, {'id': 878, 'na...",https://family.20thcenturystudios.com/movies/a...,399579.0,en,Alita: Battle Angel,...,404980543.0,122.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,An angel falls. A warrior rises.,Alita: Battle Angel,0.0,7.212,8189.0,PG-13
3,tt0441881,0.0,/7uacpbrJd9g9xSTLgfyg1NwkAiu.jpg,NaN,23934823.0,"[{'id': 10752, 'name': 'War'}, {'id': 28, 'nam...",http://dangerclosemovie.com,508664.0,en,Danger Close: The Battle of Long Tan,...,2078370.0,119.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Outnumbered. Outgunned. Never out of courage,Danger Close: The Battle of Long Tan,0.0,6.874,262.0,R
4,tt0448115,0.0,/OIGX2lm5tmlCKvZUghtwHzoxxO.jpg,"{'id': 724848, 'name': 'Shazam! Collection', '...",80000000.0,"[{'id': 28, 'name': 'Action'}, {'id': 35, 'nam...",http://www.shazammovie.com,287947.0,en,Shazam!,...,366080049.0,132.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Just say the word.,Shazam!,0.0,7.041,8535.0,PG-13


# Finding & Loading Batches of Files with GLOB

In [22]:
pd.set_option('display.max_columns',100)

In [23]:
## Checking what data we already have in our Data folder using os.listdir
import os
FOLDER = 'Data/'
file_list = sorted(os.listdir(FOLDER))
file_list

['.ipynb_checkpoints',
 'final_tmdb_data_2000.0.csv.gz',
 'final_tmdb_data_2001.0.csv.gz',
 'final_tmdb_data_2010.0.csv.gz',
 'final_tmdb_data_2011.0.csv.gz',
 'final_tmdb_data_2012.0.csv.gz',
 'final_tmdb_data_2013.0.csv.gz',
 'final_tmdb_data_2014.0.csv.gz',
 'final_tmdb_data_2015.0.csv.gz',
 'final_tmdb_data_2016.0.csv.gz',
 'final_tmdb_data_2017.0.csv.gz',
 'final_tmdb_data_2018.0.csv.gz',
 'final_tmdb_data_2019.0.csv.gz',
 'title_akas.csv.gz',
 'title_basics.csv.gz',
 'title_ratings.csv.gz',
 'tmdb_api_results_2000.0.json',
 'tmdb_api_results_2001.0.json',
 'tmdb_api_results_2010.0.json',
 'tmdb_api_results_2011.0.json',
 'tmdb_api_results_2012.0.json',
 'tmdb_api_results_2013.0.json',
 'tmdb_api_results_2014.0.json',
 'tmdb_api_results_2015.0.json',
 'tmdb_api_results_2016.0.json',
 'tmdb_api_results_2017.0.json',
 'tmdb_api_results_2018.0.json',
 'tmdb_api_results_2019.0.json',
 'tmdb_results_combined.csv.gz']

In [24]:
## let's check the filepath 
file_list[4]

'final_tmdb_data_2011.0.csv.gz'

In [25]:
## add the folder plus filename
FOLDER + file_list[4]

'Data/final_tmdb_data_2011.0.csv.gz'

In [26]:
## try read csv with folder plus filename
pd.read_csv(FOLDER + file_list[4])

,imdb_id,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,certification
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,tt0176694,0.0,/43VrmHOChLcAOG9S5tIFdsxr3u8.jpg,NaN,0.0,"[{'id': 16, 'name': 'Animation'}, {'id': 18, '...",NaN,116369.0,hu,Az ember tragédiája,Jankovics's adaptation of the eponymous play i...,2.694,/eajizCvTaeyAf2kkUfMdEgIQ5fQ.jpg,"[{'id': 55673, 'logo_path': None, 'name': 'Pan...","[{'iso_3166_1': 'HU', 'name': 'Hungary'}]",2011-11-27,0.0,166.0,"[{'english_name': 'Hungarian', 'iso_639_1': 'h...",Released,NaN,The Tragedy of Man,0.0,7.500,13.0,NaN
2,tt0210470,0.0,NaN,NaN,0.0,"[{'id': 99, 'name': 'Documentary'}]",https://www.leightonpierce.com/cms/index.php?o...,239465.0,en,50 Feet of String,"""The slow and subtle repeated rhythms of daily...",0.730,/bwuw5kBPVl8lnt1OkMZZZcuSUWz.jpg,[],"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-01-01,0.0,52.0,"[{'english_name': 'No Language', 'iso_639_1': ...",Released,NaN,50 Feet of String,0.0,6.000,3.0,NaN
3,tt0323808,0.0,/oJiBH1aSFv0gOKwb34I7CCKqgRE.jpg,NaN,7750000.0,"[{'id': 9648, 'name': 'Mystery'}, {'id': 27, '...",http://www.thewickertreemovie.com/,79544.0,en,The Wicker Tree,Director Robin Hardy's reimagining of his eeri...,3.246,/vAmjzxeMWKkQRo7Wz6mUupZ4vfN.jpg,"[{'id': 24036, 'logo_path': None, 'name': 'Eur...","[{'iso_3166_1': 'GB', 'name': 'United Kingdom'}]",2011-08-27,0.0,96.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Accept our sacrifice,The Wicker Tree,0.0,3.875,44.0,R
4,tt0372538,0.0,NaN,NaN,5000000.0,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",http://www.spythemovie.com/,323867.0,en,Spy,SPYING TURNS TO SECURITY DETAIL AS A YOUNG COU...,1.469,/fxFjBP55X6ftgXfMSrhgLodWwjp.jpg,"[{'id': 108276, 'logo_path': None, 'name': 'Sp...","[{'iso_3166_1': 'US', 'name': 'United States o...",2011-08-09,0.0,110.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,NaN,Spy,0.0,7.000,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2988,tt9282946,0.0,NaN,NaN,0.0,"[{'id': 35, 'name': 'Comedy'}]",NaN,490059.0,ko,도약선생,Won-sik is kicked out by her roommate Woo-jung...,0.600,/rXrDkS3Mpow9q915zrmiH3stAn5.jpg,[],"[{'iso_3166_1': 'KR', 'name': 'South Korea'}]",2011-06-30,0.0,65.0,"[{'english_name': 'Korean', 'iso_639_1': 'ko',...",Released,NaN,Dr. Jump,0.0,7.000,1.0,NaN
2989,tt9385434,0.0,NaN,NaN,0.0,"[{'id': 18, 'name': 'Drama'}]",NaN,566831.0,ja,お米とおっぱい。,"What would you choose, between rice and boobs,...",0.600,/4cmlMKKDkbCxm2Kjs4gfmTFL7XE.jpg,"[{'id': 11725, 'logo_path': '/3OZxd70DZ1LbVelm...","[{'iso_3166_1': 'JP', 'name': 'Japan'}]",2011-07-01,0.0,102.0,"[{'english_name': 'Japanese', 'iso_639_1': 'ja...",Released,NaN,Rice and Boobs,0.0,0.000,0.0,NaN
2990,tt9452878,0.0,NaN,NaN,0.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,108925.0,en,The Wrong Ferarri,"A presentation of surreal, somewhat connected ...",1.659,NaN,[],"[{'iso_3166_1': 'US', 'name': 'United States o...",2011-04-04,0.0,73.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,NaN,The Wrong Ferarri,0.0,5.000,2.0,NaN
2991,tt9519786,0.0,/oof2qSqrH1PAe9yEaBnId1P326G.jpg,NaN,0.0,"[{'id': 18, 'name': 'Drama'}]",NaN,874426.0,zh,North point,NaN,0.600,/ynnOmcXSIQlWnCykRWA3o0rPsAv.jpg,"[{'id': 158558, 'logo_path': None, 'name': '上海...",[],2011-11-04,0.0,0.0,[],Released,NaN,North point,0.0,0.000,0.0,NaN


In [27]:
import glob
## Make a filepath query
q = FOLDER + "*.csv.gz"
print(q)

Data/*.csv.gz


In [28]:
# Use glob.glob to get COMPLETE filepaths
file_list = glob.glob(q)
file_list

['Data\\final_tmdb_data_2000.0.csv.gz',
 'Data\\final_tmdb_data_2001.0.csv.gz',
 'Data\\final_tmdb_data_2010.0.csv.gz',
 'Data\\final_tmdb_data_2011.0.csv.gz',
 'Data\\final_tmdb_data_2012.0.csv.gz',
 'Data\\final_tmdb_data_2013.0.csv.gz',
 'Data\\final_tmdb_data_2014.0.csv.gz',
 'Data\\final_tmdb_data_2015.0.csv.gz',
 'Data\\final_tmdb_data_2016.0.csv.gz',
 'Data\\final_tmdb_data_2017.0.csv.gz',
 'Data\\final_tmdb_data_2018.0.csv.gz',
 'Data\\final_tmdb_data_2019.0.csv.gz',
 'Data\\title_akas.csv.gz',
 'Data\\title_basics.csv.gz',
 'Data\\title_ratings.csv.gz',
 'Data\\tmdb_results_combined.csv.gz']

In [30]:
# Use glob.glob to get COMPLETE filepaths and sort
q = FOLDER + "final_tmdb_data_201*.csv.gz"
print(q)
file_list = sorted(glob.glob(q))
file_list

Data/final_tmdb_data_201*.csv.gz


['Data\\final_tmdb_data_2010.0.csv.gz',
 'Data\\final_tmdb_data_2011.0.csv.gz',
 'Data\\final_tmdb_data_2012.0.csv.gz',
 'Data\\final_tmdb_data_2013.0.csv.gz',
 'Data\\final_tmdb_data_2014.0.csv.gz',
 'Data\\final_tmdb_data_2015.0.csv.gz',
 'Data\\final_tmdb_data_2016.0.csv.gz',
 'Data\\final_tmdb_data_2017.0.csv.gz',
 'Data\\final_tmdb_data_2018.0.csv.gz',
 'Data\\final_tmdb_data_2019.0.csv.gz']

In [31]:
combined_df = pd.concat([pd.read_csv(f, lineterminator = '\n') for f in file_list])
combined_df

,imdb_id,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,certification\r
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,\r
1,tt0312305,0.0,/lqUbt2cy2pnqvxKefbQAtxLS0WA.jpg,NaN,0.0,"[{'id': 10751, 'name': 'Family'}, {'id': 16, '...",http://www.qqthemovie.com/,23738.0,en,Quantum Quest: A Cassini Space Odyssey,"Dave, a young photon, who is forced out of the...",1.619,/uneDCn8y6RHKc5IAfpgznmaan9I.jpg,[],"[{'iso_3166_1': 'US', 'name': 'United States o...",2012-12-02,0.0,45.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,NaN,Quantum Quest: A Cassini Space Odyssey,0.0,7.900,8.0,\r
2,tt0326965,0.0,/xt2klJdKCVGXcoBGQrGfAS0aGDE.jpg,NaN,0.0,"[{'id': 53, 'name': 'Thriller'}, {'id': 9648, ...",http://www.inmysleep.com,40048.0,en,In My Sleep,"A sleepwalking, sexaholic masseur wakes up, co...",3.755,/sv9o6VYUQM2AbfmlTwDTn9238R5.jpg,[],"[{'iso_3166_1': 'US', 'name': 'United States o...",2010-04-23,0.0,90.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Sleepwalking can be deadly.,In My Sleep,0.0,5.500,31.0,PG-13\r
3,tt0331312,0.0,NaN,NaN,0.0,[],NaN,214026.0,en,This Wretched Life,"Awakening from a near fatal overdose, Chris em...",0.622,/xMMehQD3VWgorYoqvlTeQXSbsFo.jpg,[],[],2010-01-01,0.0,0.0,[],Released,NaN,This Wretched Life,0.0,5.000,1.0,\r
4,tt0393049,0.0,/gc9FN5zohhzCt05RkejQIIPLtBl.jpg,NaN,300000.0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,324352.0,en,Anderson's Cross,"Nick Anderson, Kevin Daniels, and Tracey Green...",1.576,/2j90os0c2HJuIjWXxMg9vMvOliw.jpg,"[{'id': 46102, 'logo_path': None, 'name': 'Ill...","[{'iso_3166_1': 'US', 'name': 'United States o...",2010-05-20,0.0,98.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Sometimes the boy next door is more than the b...,Anderson's Cross,0.0,4.000,5.0,\r
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4892,tt9906644,0.0,/htoReVItKtolso5VJqFqJDBaOwm.jpg,NaN,0.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,635903.0,ml,മനോഹരം,Manoharan is a poster artist struggling to fin...,1.448,/cNWfJlveyMEbiORrKmFKZZRDx75.jpg,"[{'id': 125107, 'logo_path': None, 'name': 'Ch...","[{'iso_3166_1': 'IN', 'name': 'India'}]",2019-09-27,0.0,122.0,"[{'english_name': 'Malayalam', 'iso_639_1': 'm...",Released,NaN,Manoharam,0.0,6.125,12.0,\r
4893,tt9913660,0.0,/z6YpkQiWLnRKl5yQD9dSiPDFWN.jpg,NaN,0.0,"[{'id': 18, 'name': 'Drama'}]",NaN,711261.0,en,No Apology,When a group of women struggle to deal with th...,0.638,/fqIWD97MAjgrj8mB2M78kxeSSnk.jpg,[],[],2019-10-23,0.0,102.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,NaN,No Apology,0.0,8.000,1.0,\r
4894,tt9913872,0.0,/a0kySBmMZ77AqmNA3pDu45GsbCp.jpg,NaN,0.0,"[{'id': 27, 'name': 'Horror'}]",NaN,800030.0,es,De la piel del Diablo,NaN,0.600,/it2z2X2pxeurP1Cmzo5C3BJdQ2W.jpg,[],[],2019-01-01,0.0,0.0,[],Released,NaN,De la piel del Diablo,0.0,8.200,3.0,\r
4895,tt9914942,0.0,/tjHUeULyw2vtS1DXFp0OHfhUzRX.jpg,NaN,0.0,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",NaN,598553.0,ca,La vida sense la Sara Amat,"Pep, a 13-year-old boy, is in love with a girl...",5.570,/ulCLv5kRiC8JnxdRBczDzx0NjSr.jpg,"[{'id': 20786, 'logo_path': None, 'name': ""Mas...","[{'iso_3166_1': 'ES', 'name': 'Spain'}]",2019-07-12,0.0,74.0,"[{'english_name': 'Catalan', 'iso_639_1': 'ca'...",Released,NaN,Life without Sara Amat,0.0,7.700,7.0,\r


In [32]:
# remove ids that are 0  and then reset index
combined_df = combined_df.loc[combined_df['imdb_id']!='0']
combined_df.head()

,imdb_id,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,certification\r
1,tt0312305,0.0,/lqUbt2cy2pnqvxKefbQAtxLS0WA.jpg,NaN,0.0,"[{'id': 10751, 'name': 'Family'}, {'id': 16, '...",http://www.qqthemovie.com/,23738.0,en,Quantum Quest: A Cassini Space Odyssey,"Dave, a young photon, who is forced out of the...",1.619,/uneDCn8y6RHKc5IAfpgznmaan9I.jpg,[],"[{'iso_3166_1': 'US', 'name': 'United States o...",2012-12-02,0.0,45.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,NaN,Quantum Quest: A Cassini Space Odyssey,0.0,7.90,8.0,\r
2,tt0326965,0.0,/xt2klJdKCVGXcoBGQrGfAS0aGDE.jpg,NaN,0.0,"[{'id': 53, 'name': 'Thriller'}, {'id': 9648, ...",http://www.inmysleep.com,40048.0,en,In My Sleep,"A sleepwalking, sexaholic masseur wakes up, co...",3.755,/sv9o6VYUQM2AbfmlTwDTn9238R5.jpg,[],"[{'iso_3166_1': 'US', 'name': 'United States o...",2010-04-23,0.0,90.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Sleepwalking can be deadly.,In My Sleep,0.0,5.50,31.0,PG-13\r
3,tt0331312,0.0,NaN,NaN,0.0,[],NaN,214026.0,en,This Wretched Life,"Awakening from a near fatal overdose, Chris em...",0.622,/xMMehQD3VWgorYoqvlTeQXSbsFo.jpg,[],[],2010-01-01,0.0,0.0,[],Released,NaN,This Wretched Life,0.0,5.00,1.0,\r
4,tt0393049,0.0,/gc9FN5zohhzCt05RkejQIIPLtBl.jpg,NaN,300000.0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,324352.0,en,Anderson's Cross,"Nick Anderson, Kevin Daniels, and Tracey Green...",1.576,/2j90os0c2HJuIjWXxMg9vMvOliw.jpg,"[{'id': 46102, 'logo_path': None, 'name': 'Ill...","[{'iso_3166_1': 'US', 'name': 'United States o...",2010-05-20,0.0,98.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Sometimes the boy next door is more than the b...,Anderson's Cross,0.0,4.00,5.0,\r
5,tt0398286,0.0,/cWczNud8Y8i8ab0Z4bxos4myWYO.jpg,NaN,260000000.0,"[{'id': 16, 'name': 'Animation'}, {'id': 10751...",http://disney.go.com/disneypictures/tangled/,38757.0,en,Tangled,When the kingdom's most wanted-and most charmi...,108.562,/ym7Kst6a4uodryxqbGOxmewF235.jpg,"[{'id': 6125, 'logo_path': '/tzsMJBJZINu7GHzrp...","[{'iso_3166_1': 'US', 'name': 'United States o...",2010-11-24,592461732.0,100.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,They're taking adventure to new lengths.,Tangled,0.0,7.59,10223.0,PG\r


In [33]:
combined_df = combined_df.reset_index(drop = True)

In [34]:
## saving the combined csv to disk
combined_df.to_csv(FOLDER+'combined_tmdb_data.csv.gz',compression='gzip',index=False)

combined_df = pd.read_csv(FOLDER+'combined_tmdb_data.csv.gz',lineterminator='\n')
combined_df.head()

,imdb_id,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,certification\r\r
0,tt0312305,0.0,/lqUbt2cy2pnqvxKefbQAtxLS0WA.jpg,NaN,0.0,"[{'id': 10751, 'name': 'Family'}, {'id': 16, '...",http://www.qqthemovie.com/,23738.0,en,Quantum Quest: A Cassini Space Odyssey,"Dave, a young photon, who is forced out of the...",1.619,/uneDCn8y6RHKc5IAfpgznmaan9I.jpg,[],"[{'iso_3166_1': 'US', 'name': 'United States o...",2012-12-02,0.0,45.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,NaN,Quantum Quest: A Cassini Space Odyssey,0.0,7.90,8.0,\r\r
1,tt0326965,0.0,/xt2klJdKCVGXcoBGQrGfAS0aGDE.jpg,NaN,0.0,"[{'id': 53, 'name': 'Thriller'}, {'id': 9648, ...",http://www.inmysleep.com,40048.0,en,In My Sleep,"A sleepwalking, sexaholic masseur wakes up, co...",3.755,/sv9o6VYUQM2AbfmlTwDTn9238R5.jpg,[],"[{'iso_3166_1': 'US', 'name': 'United States o...",2010-04-23,0.0,90.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Sleepwalking can be deadly.,In My Sleep,0.0,5.50,31.0,PG-13\r\r
2,tt0331312,0.0,NaN,NaN,0.0,[],NaN,214026.0,en,This Wretched Life,"Awakening from a near fatal overdose, Chris em...",0.622,/xMMehQD3VWgorYoqvlTeQXSbsFo.jpg,[],[],2010-01-01,0.0,0.0,[],Released,NaN,This Wretched Life,0.0,5.00,1.0,\r\r
3,tt0393049,0.0,/gc9FN5zohhzCt05RkejQIIPLtBl.jpg,NaN,300000.0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,324352.0,en,Anderson's Cross,"Nick Anderson, Kevin Daniels, and Tracey Green...",1.576,/2j90os0c2HJuIjWXxMg9vMvOliw.jpg,"[{'id': 46102, 'logo_path': None, 'name': 'Ill...","[{'iso_3166_1': 'US', 'name': 'United States o...",2010-05-20,0.0,98.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Sometimes the boy next door is more than the b...,Anderson's Cross,0.0,4.00,5.0,\r\r
4,tt0398286,0.0,/cWczNud8Y8i8ab0Z4bxos4myWYO.jpg,NaN,260000000.0,"[{'id': 16, 'name': 'Animation'}, {'id': 10751...",http://disney.go.com/disneypictures/tangled/,38757.0,en,Tangled,When the kingdom's most wanted-and most charmi...,108.562,/ym7Kst6a4uodryxqbGOxmewF235.jpg,"[{'id': 6125, 'logo_path': '/tzsMJBJZINu7GHzrp...","[{'iso_3166_1': 'US', 'name': 'United States o...",2010-11-24,592461732.0,100.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,They're taking adventure to new lengths.,Tangled,0.0,7.59,10223.0,PG\r\r
